In [99]:
import numpy as np 
import re

In [100]:
L = 32 
RA = 4
RB = 2
RC = 2
PA = 2
PB = 2
#PC = 1 so fuck it we guuci 

edge_re = re.compile(
    r"\((\d+),(\d+),(\d+)\)x"
    r"\((\d+),(\d+),(\d+)\)\s*->\s*"
    r"\((\d+),(\d+)\)"
)

def parse_edges(filename):
    edges = []

    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            m = edge_re.match(line)
            if not m:
                raise ValueError(f"Could not parse line:\n{line}")

            la, ra, pa, lb, rb, pb, lc, rc = map(int, m.groups())

            A = (la, ra, pa)
            B = (lb, rb, pb)
            C = (lc, rc)

            edges.append((A, B, C))

    return edges


In [101]:
edges = parse_edges("probe_data.txt")

In [102]:
edges
#are the triples of a_coord, b_coord that hit lane x in the c_coordinate, 
#always also have lane x? that is is data redistributed among lanes after the 
#tensor core does the mma? 


[((0, 0, 0), (0, 0, 0), (0, 0)),
 ((0, 0, 0), (4, 0, 0), (0, 1)),
 ((0, 0, 0), (8, 0, 0), (1, 0)),
 ((0, 0, 0), (12, 0, 0), (1, 1)),
 ((0, 0, 0), (16, 0, 0), (2, 0)),
 ((0, 0, 0), (20, 0, 0), (2, 1)),
 ((0, 0, 0), (24, 0, 0), (3, 0)),
 ((0, 0, 0), (28, 0, 0), (3, 1)),
 ((0, 0, 1), (0, 0, 1), (0, 0)),
 ((0, 0, 1), (4, 0, 1), (0, 1)),
 ((0, 0, 1), (8, 0, 1), (1, 0)),
 ((0, 0, 1), (12, 0, 1), (1, 1)),
 ((0, 0, 1), (16, 0, 1), (2, 0)),
 ((0, 0, 1), (20, 0, 1), (2, 1)),
 ((0, 0, 1), (24, 0, 1), (3, 0)),
 ((0, 0, 1), (28, 0, 1), (3, 1)),
 ((0, 1, 0), (0, 0, 0), (0, 2)),
 ((0, 1, 0), (4, 0, 0), (0, 3)),
 ((0, 1, 0), (8, 0, 0), (1, 2)),
 ((0, 1, 0), (12, 0, 0), (1, 3)),
 ((0, 1, 0), (16, 0, 0), (2, 2)),
 ((0, 1, 0), (20, 0, 0), (2, 3)),
 ((0, 1, 0), (24, 0, 0), (3, 2)),
 ((0, 1, 0), (28, 0, 0), (3, 3)),
 ((0, 1, 1), (0, 0, 1), (0, 2)),
 ((0, 1, 1), (4, 0, 1), (0, 3)),
 ((0, 1, 1), (8, 0, 1), (1, 2)),
 ((0, 1, 1), (12, 0, 1), (1, 3)),
 ((0, 1, 1), (16, 0, 1), (2, 2)),
 ((0, 1, 1), (20, 0, 1), (

In [103]:
def get_A_B_coords_by_C_lane(edges): 
  group_by_c_lane = {i:[] for i in range(32)}
  for e in edges: 
    a_coord,b_coord, c_coord = e
    group_by_c_lane[c_coord[0]].append(e)
  return group_by_c_lane  

In [104]:
group_by_c_lane_dict = get_A_B_coords_by_C_lane(edges)

In [105]:
def get_C_inv_map(edges):
    C_inv_map = {c_coord: [] for _, _, c_coord in edges}

    for a, b, c in edges:
        C_inv_map[c].append((a, b))

    # sort each list lexicographically
    for c in C_inv_map:
        C_inv_map[c].sort()

    return C_inv_map
  

In [106]:
C_inv_map = get_C_inv_map(edges)

In [107]:
print(C_inv_map[0,0])
print(C_inv_map[0,1])
print(C_inv_map[0,2])
print(C_inv_map[0,3])

[((0, 0, 0), (0, 0, 0)), ((0, 0, 1), (0, 0, 1)), ((0, 2, 0), (0, 1, 0)), ((0, 2, 1), (0, 1, 1)), ((1, 0, 0), (1, 0, 0)), ((1, 0, 1), (1, 0, 1)), ((1, 2, 0), (1, 1, 0)), ((1, 2, 1), (1, 1, 1)), ((2, 0, 0), (2, 0, 0)), ((2, 0, 1), (2, 0, 1)), ((2, 2, 0), (2, 1, 0)), ((2, 2, 1), (2, 1, 1)), ((3, 0, 0), (3, 0, 0)), ((3, 0, 1), (3, 0, 1)), ((3, 2, 0), (3, 1, 0)), ((3, 2, 1), (3, 1, 1))]
[((0, 0, 0), (4, 0, 0)), ((0, 0, 1), (4, 0, 1)), ((0, 2, 0), (4, 1, 0)), ((0, 2, 1), (4, 1, 1)), ((1, 0, 0), (5, 0, 0)), ((1, 0, 1), (5, 0, 1)), ((1, 2, 0), (5, 1, 0)), ((1, 2, 1), (5, 1, 1)), ((2, 0, 0), (6, 0, 0)), ((2, 0, 1), (6, 0, 1)), ((2, 2, 0), (6, 1, 0)), ((2, 2, 1), (6, 1, 1)), ((3, 0, 0), (7, 0, 0)), ((3, 0, 1), (7, 0, 1)), ((3, 2, 0), (7, 1, 0)), ((3, 2, 1), (7, 1, 1))]
[((0, 1, 0), (0, 0, 0)), ((0, 1, 1), (0, 0, 1)), ((0, 3, 0), (0, 1, 0)), ((0, 3, 1), (0, 1, 1)), ((1, 1, 0), (1, 0, 0)), ((1, 1, 1), (1, 0, 1)), ((1, 3, 0), (1, 1, 0)), ((1, 3, 1), (1, 1, 1)), ((2, 1, 0), (2, 0, 0)), ((2, 1, 1), (

In [108]:
def T_lb_plus_4(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra, pa), (lb + 4, rb, pb)

def T_ra_plus_1(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra + 1, pa), (lb, rb, pb)

def T_both(a, b):
    la, ra, pa = a
    lb, rb, pb = b
    return (la, ra + 1, pa), (lb + 4, rb, pb)
  
  
def normalize(pairs):
    return set(pairs)

def test_rc_invariant(C_inv_map, lc=0):
    base = normalize(C_inv_map[(lc, 0)])

    tests = {
        1: lambda p: T_lb_plus_4(*p),
        2: lambda p: T_ra_plus_1(*p),
        3: lambda p: T_both(*p),
    }

    for rc, T in tests.items():
        transformed = set(T((a,b)) for (a,b) in base)
        target = normalize(C_inv_map[(lc, rc)])

        if transformed != target:
            print(f"❌ invariant failed for rc = {rc}")
            print("missing:", transformed - target)
            print("extra:", target - transformed)
            return False

    print("✅ rc-invariant holds for lc =", lc)
    return True



In [109]:
for i in range(32):
  test_rc_invariant(C_inv_map, lc=i)

✅ rc-invariant holds for lc = 0
✅ rc-invariant holds for lc = 1
✅ rc-invariant holds for lc = 2
✅ rc-invariant holds for lc = 3
✅ rc-invariant holds for lc = 4
✅ rc-invariant holds for lc = 5
✅ rc-invariant holds for lc = 6
✅ rc-invariant holds for lc = 7
✅ rc-invariant holds for lc = 8
✅ rc-invariant holds for lc = 9
✅ rc-invariant holds for lc = 10
✅ rc-invariant holds for lc = 11
✅ rc-invariant holds for lc = 12
✅ rc-invariant holds for lc = 13
✅ rc-invariant holds for lc = 14
✅ rc-invariant holds for lc = 15
✅ rc-invariant holds for lc = 16
✅ rc-invariant holds for lc = 17
✅ rc-invariant holds for lc = 18
✅ rc-invariant holds for lc = 19
✅ rc-invariant holds for lc = 20
✅ rc-invariant holds for lc = 21
✅ rc-invariant holds for lc = 22
✅ rc-invariant holds for lc = 23
✅ rc-invariant holds for lc = 24
✅ rc-invariant holds for lc = 25
✅ rc-invariant holds for lc = 26
✅ rc-invariant holds for lc = 27
✅ rc-invariant holds for lc = 28
✅ rc-invariant holds for lc = 29
✅ rc-invariant holds

In [110]:
def get_AxB_to_C(edges): 
  forward_rel = {(a,b):[] for a,b,_ in edges}
  for a,b,c in edges: 
    forward_rel[(a,b)].append(c)
    
  return forward_rel
  
    

In [111]:
forward_rel = get_AxB_to_C(edges)

In [112]:
for la in range(L):
  for lb in range(L): 
    for ra in range(RA): 
      for rb in range(RB):
        for pa in range(PA): 
          for pb in range(PB): 
            x = ((la,ra,pa),(lb,rb,pb))
            if x in forward_rel: 
              image_x = forward_rel[x]
              #print(len(image_x)==1)

In [113]:
C_inv_map[0,0]

[((0, 0, 0), (0, 0, 0)),
 ((0, 0, 1), (0, 0, 1)),
 ((0, 2, 0), (0, 1, 0)),
 ((0, 2, 1), (0, 1, 1)),
 ((1, 0, 0), (1, 0, 0)),
 ((1, 0, 1), (1, 0, 1)),
 ((1, 2, 0), (1, 1, 0)),
 ((1, 2, 1), (1, 1, 1)),
 ((2, 0, 0), (2, 0, 0)),
 ((2, 0, 1), (2, 0, 1)),
 ((2, 2, 0), (2, 1, 0)),
 ((2, 2, 1), (2, 1, 1)),
 ((3, 0, 0), (3, 0, 0)),
 ((3, 0, 1), (3, 0, 1)),
 ((3, 2, 0), (3, 1, 0)),
 ((3, 2, 1), (3, 1, 1))]

In [114]:
for x,y in zip(C_inv_map[3,1], C_inv_map[4,1]):
  print(x[1] == y[1])
  print(x[1],y[1])

  

False
(28, 0, 0) (4, 0, 0)
False
(28, 0, 1) (4, 0, 1)
False
(28, 1, 0) (4, 1, 0)
False
(28, 1, 1) (4, 1, 1)
False
(29, 0, 0) (5, 0, 0)
False
(29, 0, 1) (5, 0, 1)
False
(29, 1, 0) (5, 1, 0)
False
(29, 1, 1) (5, 1, 1)
False
(30, 0, 0) (6, 0, 0)
False
(30, 0, 1) (6, 0, 1)
False
(30, 1, 0) (6, 1, 0)
False
(30, 1, 1) (6, 1, 1)
False
(31, 0, 0) (7, 0, 0)
False
(31, 0, 1) (7, 0, 1)
False
(31, 1, 0) (7, 1, 0)
False
(31, 1, 1) (7, 1, 1)


In [115]:
def transform_base_to(lc, rc, a, b):
    la, ra, pa = a
    lb, rb, pb = b

    a2 = (
        la + 4 * (lc // 4),
        ra + (rc // 2),
        pa
    )

    b2 = (
        lb + 4 * (rc % 2) + 8 * (lc % 4),
        rb,
        pb
    )

    return a2, b2

def transform_to_base(lc, rc, a, b):
    la, ra, pa = a
    lb, rb, pb = b

    a0 = (
        la - 4 * (lc // 4),
        ra - (rc // 2),
        pa
    )

    b0 = (
        lb - 4 * (rc % 2) - 8 * (lc % 4),
        rb,
        pb
    )

    return a0, b0

edge_set = set(edges)

base_edges = set(
    (a, b)
    for (a, b, c) in edges
    if c == (0, 0)
)

def check_forward(edges):
    edge_set = set(edges)

    for a0, b0 in base_edges:
        for lc in range(L):
            for rc in range(4):
                a2, b2 = transform_base_to(lc, rc, a0, b0)

                if (a2, b2, (lc, rc)) not in edge_set:
                    print("❌ forward failed")
                    print("base:", a0, b0)
                    print("target:", a2, b2, (lc, rc))
                    return False

    print("✅ forward transform holds")
    return True
  
def check_backward(edges):
    edge_set = set(edges)

    for (a, b, (lc, rc)) in edges:
        a0, b0 = transform_to_base(lc, rc, a, b)

        if (a0, b0) not in base_edges:
            print("❌ backward failed")
            print("edge:", a, b, (lc, rc))
            print("maps to base:", a0, b0)
            return False

    print("✅ backward transform holds")
    return True



In [116]:
check_forward(edges)
check_backward(edges)


✅ forward transform holds
✅ backward transform holds


True

In [120]:
C_inv_map[0,0]

[((0, 0, 0), (0, 0, 0)),
 ((0, 0, 1), (0, 0, 1)),
 ((0, 2, 0), (0, 1, 0)),
 ((0, 2, 1), (0, 1, 1)),
 ((1, 0, 0), (1, 0, 0)),
 ((1, 0, 1), (1, 0, 1)),
 ((1, 2, 0), (1, 1, 0)),
 ((1, 2, 1), (1, 1, 1)),
 ((2, 0, 0), (2, 0, 0)),
 ((2, 0, 1), (2, 0, 1)),
 ((2, 2, 0), (2, 1, 0)),
 ((2, 2, 1), (2, 1, 1)),
 ((3, 0, 0), (3, 0, 0)),
 ((3, 0, 1), (3, 0, 1)),
 ((3, 2, 0), (3, 1, 0)),
 ((3, 2, 1), (3, 1, 1))]

In [131]:
def C_inv_0_0_generator(i):
  assert 0 <= i < 16
  
  la,lb = i // 4, i // 4
  ra,rb = 2*((i//2)%2),((i//2)%2)
  pa,pb = i % 2, i % 2
  return ((la,ra,pa),(lb,rb,pb)) 

In [132]:
def generate_C_inv_0_0():
  gen = []
  for i in range(16):
    gen.append(C_inv_0_0_generator(i))
    
  return set(gen)

In [133]:
print(set(C_inv_map[0,0]) == generate_C_inv_0_0())

True


In [135]:
#finding the predicate: 
for a,b,c in edges: 
  print(a[2] == b[2])
  print(a[1] == 2*b[1])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
True
False
Tru

In [140]:
def generate_inv_map_by_k(k,lc,rc):
  la = (k//4) + (4*(lc//4))
  lb = (k//4) + (8*(lc % 4)) + (4*(rc % 2))
  ra = (2*((k//2)%2)) + ((rc//2))
  rb = (k//2)%2
  pa = k % 2
  pb = k % 2
  return((la,ra,pa),(lb,rb,pb))
  
def generate_inv_map(lc,rc): 
  L = []
  for k in range(16):
    L.append(generate_inv_map_by_k(k,lc,rc))
    
  return set(L)

def verify(C_inv_map): 
  for lc in range(L): 
    for rc in range(RC): 
      inv_map = set(C_inv_map[lc,rc])
      generated_inv_map = generate_inv_map(lc,rc)
      if inv_map != generated_inv_map:
        return False
      
  return True

In [141]:
verify(C_inv_map)

True